In [40]:
import getpass
import os

from dotenv import load_dotenv

# Load the .env file
_ = load_dotenv()

In [41]:
from agno.models.huggingface import HuggingFace
from agno.embedder.ollama import OllamaEmbedder
from agno.models.ollama import Ollama
from agno.agent import Agent
from agno.tools.duckduckgo import DuckDuckGoTools



* LLM

In [42]:
model=HuggingFace(
        id="meta-llama/Meta-Llama-3-8B-Instruct",
        api_key=os.getenv("HUGGINGFACE_API_TOKEN"),
)





In [43]:
ollama_embedder = OllamaEmbedder(id="nomic-embed-text:latest",dimensions=768)

### Web Searcher Agent

In [44]:
web_searcher = Agent(model=model, 
                     tools=[DuckDuckGoTools()], 
                     name="Web Searcher",  
                     role="Searches the web for information on a topic", 
                     show_tool_calls=True,
                     markdown=False)

response = web_searcher.run("Who is Thomas Edison?")
print(response.content) #latencia 1min 50

Thomas Edison was an American inventor and businessman who developed many important technologies. He is best known for his invention of the light bulb, phonograph, and motion picture camera.


###  RAG Agent

In [45]:
from agno.vectordb.lancedb import LanceDb, SearchType

# Define the database URL where the vector database will be stored
db_url = "./tmp/lancedb"
vector_db = LanceDb(
    table_name="recipes",  # Table name in the vector database
    uri=db_url,  # Location to initiate/create the vector database
    embedder=ollama_embedder,  # Without using this, it will use OpenAIChat embeddings by default
    search_type=SearchType.vector,
)

In [46]:
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
# Create a knowledge base of PDFs from URLs
knowledge_base = PDFUrlKnowledgeBase(
    urls=["https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
    vector_db=vector_db
)
# Load the knowledge base
knowledge_base.load(recreate=False)

INFO     Creating collection

INFO     Loading knowledge base

INFO     Reading: https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf

INFO     Added 0 documents to knowledge base

In [47]:
# Create and use the agent
rag_agent = Agent(model=model,
              name="RAG Agent",
              role="Responds to questions about Thai recipes", 
              knowledge=knowledge_base,  

              # Add a tool to search the knowledge base which enables agentic RAG.
               # This is enabled by default when `knowledge` is provided to the Agent.
              search_knowledge=True,
              show_tool_calls=True,
              markdown=False)

In [48]:

response = rag_agent.run("How do I make chicken and galangal in coconut milk soup")
print("response:", response.content)

response: To make chicken and galangal in coconut milk soup, you will need the following ingredients: 1 pound boneless, skinless chicken breast, 3 cloves of garlic, 1 tablespoon grated ginger, 1 tablespoon grated galangal, 2 cups coconut milk, 2 cups chicken broth, 1 tablespoon fish sauce, 1 tablespoon lime juice, 1 teaspoon palm sugar, 1/4 teaspoon turmeric, 1/4 teaspoon paprika, salt, and pepper. Instructions: 1. In a medium saucepan, heat the coconut milk and chicken broth over medium heat until it starts to simmer. 2. Add the chicken, garlic, ginger, galangal, fish sauce, lime juice, palm sugar, turmeric, paprika, salt, and pepper. Stir to combine. 3. Reduce the heat to low and let the soup simmer for 20-25 minutes or until the chicken is cooked through. 4. Serve the soup hot and garnish with fresh herbs like cilantro and lime wedges. You can adjust the level of spiciness to your liking by adding more or less chili peppers.


### Workflow

In [58]:
from agno.agent import Agent, RunResponse
from agno.workflow import Workflow
from pydantic import BaseModel, Field
import json
from typing import Optional, Iterator
from agno.models.ollama import Ollama

ollama_model=Ollama(id="llama3.2:3b")

class NewsArticle(BaseModel):
    title: str = Field(..., description="Title of the article.")
    url: str = Field(..., description="Link to the article.")
    summary: Optional[str] = Field(..., description="Summary of the article if available.")

class SearchResults(BaseModel):
    articles: list[NewsArticle]

class BlogPostGenerator(Workflow):
    # Define an Agent that will search the web for a topic
    searcher_agent: Agent = Agent(
        model=ollama_model,
        tools=[DuckDuckGoTools()],
        instructions=["Given a topic, search for the top 1 articles."],
        response_model=SearchResults,
        structured_outputs=True,
    )

    # Define an Agent that will write the blog post
    writer_agent: Agent = Agent(
        model=ollama_model,
        instructions=[
            "You will be provided with a topic and a list of top articles on that topic.",
            "Carefully read each article and generate a New York Times worthy blog post on that topic.",
            "Break the blog post into sections and provide key takeaways at the end.",
            "Make sure the title is catchy and engaging.",
            "Always provide sources, do not make up information or sources.",
        ],
        markdown=True,
    )

    def run(self, topic: str) -> Iterator[RunResponse]:
        """This is where the main logic of the workflow is implemented."""

        # Step 1: Search the web for articles on the topic
        search_results: Optional[SearchResults] = self._get_search_results(topic)
        if search_results is None:
            anwser=f"Sorry, could not find any articles on the topic: {topic}",
        else:
            # Step 2: Write the blog post
            anwser = self._write_blog_post(topic, search_results)
            
        return anwser
                                                                           

    def _get_search_results(self, topic: str) -> Optional[SearchResults]:
        """Get the search results for a topic."""
        print("Calling _get_search_results method")
        try:
            searcher_response: RunResponse = self.searcher_agent.run(topic)
            return searcher_response.content
        
        except Exception as e:
            print(f"Error getting search results for topic {topic}: {e}")
            return None
     
    def _write_blog_post(self,topic, search_results)-> Iterator[RunResponse]:
        """Write a blog post on a topic."""
        print("Calling _write_blog_post method")
        writer_input = {"topic": topic, "articles": [v.model_dump() for v in search_results.articles]}
        # Run the writer and yield the response
        return self.writer_agent.run(json.dumps(writer_input, indent=4))


In [59]:
# flow = BlogPostGenerator()
# response = flow.run(topic="What is artificial intelligence?")
# response

Calling _get_search_results method


WARNING  Error retrieving structured outputs: 'Message' object has no attribute 'parsed'

Calling _write_blog_post method


In [51]:
hn_team = Agent(
    name="Multi-Agent Team",
    model=model,
    team=[rag_agent, web_searcher],
    instructions=[
        "First, search hackernews for what the user is asking about.",
        "Then, ask the article reader to read the links for the stories to get more information.",
        "Important: you must provide the article reader with the links to read.",
        "Then, ask the web searcher to search for each story to get more information.",
        "Finally, provide a thoughtful and engaging summary.",
    ],
    show_tool_calls=True,
    markdown=True,
)
hn_team.print_response("Write an article about the top 2 stories on hackernews", stream=True)

UnknownError: {'message': "Input validation error: cannot compile regex from schema: 'type' must be a string", 'http_status_code': 422}